In [2]:
import DataLoader_tensor_sparse as dl
import SCDataset as ds
import anndata as ad
import numpy as np
import torch
from torch.utils.data import TensorDataset

### Reading the h5ad file

In [2]:
adata_subset = ad.read_h5ad('Larry_41201_2000.h5ad')
print("adata_subset.X.shape:", adata_subset.X.shape)

adata_subset.X.shape: (41201, 2000)


### Cell-pairs for trainning contrastive learning

In [3]:
#----------------------------------------------------Generate Batches--------------------------------------------------------
# input data
count_matrix = adata_subset.X
cell_lineage = adata_subset.obs['clone_id'].values.reshape(-1, 1)
# count_matrix.shape, cell_lineage.shape

# step 1 generate designed batches
# batchsize = 10
DLoader = dl.SClineage_DataLoader(count_matrix,cell_lineage,batch_size= 10, seed=7)
batch_all, num_batch, lineage_info = DLoader.batch_generator()

larry_dataset = ds.SCDataset(batches=batch_all)

print("number of batches: ", num_batch)

number of batches:  2154


In [4]:
data_loader = torch.utils.data.DataLoader(dataset=larry_dataset, batch_size=10, shuffle=False, num_workers=1) 

In [5]:
i = 1
for batch in data_loader: 
    X, Y = batch
    if i ==1:
        print(X.shape)
    i+=1

torch.Size([10, 2000])


### All cells for getting the reprsentation

In [6]:
count_matrix_arr = count_matrix.toarray()
count_matrix_th = torch.from_numpy(count_matrix_arr)
dataset = TensorDataset(count_matrix_th)
data_loader_all = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=False, num_workers=1,drop_last=False)
print("num of batches for all cells (not cell pairs):", len(data_loader_all))

num of batches for all cells (not cell pairs): 4121


In [7]:
last_batch_data = None

for data in data_loader_all:
    last_batch_data = data

last_batch_data[0].shape

In [11]:
i= 0 
for batch in data_loader_all: 
    X= batch[0]
    if i == 0:
        print(X.shape)
    i+=1

torch.Size([10, 2000])


In [3]:
labels = np.load("out/lineage_info.npy")
len(labels)

21540